## 使い方

無料のTesla T4 Google Colabインスタンスでこのノートブックを実行するには:
1. 上部メニューから「**Runtime**」（ランタイム）をクリック
2. 「**Run all**」（すべて実行）を選択

すると、全てのセルが順番に実行され、モデルのファインチューニングが完了します。

---

### Unslothについて

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> 

困ったときはDiscordに参加してください。また、<a href="https://github.com/unslothai/unsloth">Github</a>でスターをお願いします！
</div>

自分のコンピューターにUnslothをインストールするには、Githubページの[インストール手順](https://docs.unsloth.ai/get-started/installing-+-updating)を参照してください。

このノートブックでは以下を学びます:
- [データの準備](#Data)
- [モデルの学習](#Train)
- [モデルの実行（推論）](#Inference)
- [モデルの保存](#Save)

## 最新情報

Unslothの最新アップデート情報

**[Qwen3ガイド](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)**と、他の量子化手法を上回る新しい**[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)**量子化をチェックしてください！

すべての[モデルアップロード](https://docs.unsloth.ai/get-started/all-our-models)と[ノートブック](https://docs.unsloth.ai/get-started/unsloth-notebooks)については、ドキュメントを参照してください。

## ステップ1: インストール

必要なライブラリをインストールします。このセルは自動的にGoogle Colab環境を検出し、適切なインストールを行います。

### Installation

## ステップ2: Unslothでモデルを読み込む

ここでは、**Llama 3.1 8B**モデルを読み込みます。Unslothを使用することで、メモリ効率的に大規模モデルを扱えます。

### Unsloth

## ステップ3: LoRAアダプターの追加

**LoRA**（Low-Rank Adaptation）を使用することで、全パラメータの1〜10%のみを更新するだけで効果的なファインチューニングが可能になります。

In [ ]:
# --- 必要なライブラリをインポート ---
from unsloth import FastLanguageModel  # Unslothの高速言語モデルクラス
import torch  # PyTorchフレームワーク

# --- モデルの設定 ---
# 最大シーケンス長（入力+出力の最大トークン数）
# 任意の値を選択可能（RoPE Scalingが自動的にサポートされます）
max_seq_length = 2048

# データ型の設定（None = 自動検出）
# Float16: Tesla T4、V100用
# Bfloat16: Ampere世代以降のGPU用
dtype = None

# 4bit量子化を使用するかどうか
# True: メモリ使用量を大幅に削減（推奨）
# False: 完全な精度だがメモリ消費が大きい
load_in_4bit = True

# --- サポートされている4bit事前量子化モデルのリスト ---
# これらのモデルは4倍高速にダウンロードでき、メモリ不足(OOM)を防ぎます
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15兆トークンで学習、2倍高速！
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",  # Instruct版（指示に従うように調整済み）
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",     # 70Bパラメータの大規模版
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 405B版も4bitで利用可能！
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # 新しいMistral 12b、2倍高速！
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3、2倍高速！
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5、2倍高速！
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma、2倍高速！
]  # その他のモデルは https://huggingface.co/unsloth を参照

# --- モデルとトークナイザーを読み込む ---
# FastLanguageModel.from_pretrained()は、事前学習済みモデルを効率的に読み込む
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # 使用するモデル名
    max_seq_length = max_seq_length,  # 最大シーケンス長
    dtype = dtype,  # データ型（自動検出）
    load_in_4bit = load_in_4bit,  # 4bit量子化を有効化
    # token = "hf_...",  # ゲート付きモデル（meta-llama/Llama-2-7b-hfなど）を使用する場合に必要
)

## ステップ4: データの準備 {#Data}

ここでは、[yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned)が提供するAlpacaデータセット（52Kサンプル）を使用します。これは、元の[Alpacaデータセット](https://crfm.stanford.edu/2023/03/13/alpaca.html)をフィルタリングしたものです。

**注意**: このコードセクションは、独自のデータに置き換えることができます。

### 重要なポイント

1. **completion（応答部分）のみを学習する場合**: TRLのドキュメント[こちら](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)を参照してください。

2. **EOS_TOKEN（終了トークン）を必ず追加**: トークン化した出力にEOS_TOKENを追加しないと、生成が無限に続いてしまいます。

3. **ShareGPTデータセット用のllama-3テンプレート**: [会話型ノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)を試してください。

4. **小説執筆などのテキスト補完**: [テキスト補完ノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb)を試してください。

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

## ステップ5: モデルの学習 {#Train}

ここでは、Hugging FaceのTRLの`SFTTrainer`を使用します。詳細は[TRL SFTドキュメント](https://huggingface.co/docs/trl/sft_trainer)を参照してください。

このノートブックでは、高速化のために60ステップのみ実行しますが、完全な学習を行う場合は`num_train_epochs=1`を設定し、`max_steps=None`にしてください。

**注意**: UnslothはTRLの`DPOTrainer`（人間のフィードバックによる最適化）もサポートしています！

In [ ]:
# --- LoRAアダプターをモデルに追加 ---
# LoRA (Low-Rank Adaptation): モデル全体ではなく、小さなアダプター層のみを学習することで
# メモリと時間を大幅に節約する効率的なファインチューニング手法

model = FastLanguageModel.get_peft_model(
    model,
    
    # --- LoRAのランク（r） ---
    # アダプター行列のランク。大きいほど表現力が高いが、メモリ消費が増える
    # 推奨値: 8, 16, 32, 64, 128（0より大きい任意の数を選択可能）
    r = 16,
    
    # --- ターゲットモジュール ---
    # LoRAを適用するレイヤーのリスト
    # q_proj, k_proj, v_proj: Attentionの Query, Key, Value 射影層
    # o_proj: Attentionの出力射影層
    # gate_proj, up_proj, down_proj: Feed-Forwardネットワークの層
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    
    # --- LoRAアルファ ---
    # LoRAのスケーリングファクター（通常はrと同じ値に設定）
    lora_alpha = 16,
    
    # --- LoRAドロップアウト ---
    # 過学習を防ぐためのドロップアウト率
    # 0 = ドロップアウトなし（最適化されているため推奨）
    lora_dropout = 0,
    
    # --- バイアス ---
    # バイアス項の学習方法
    # "none": バイアスを学習しない（最適化されているため推奨）
    # その他: "all", "lora_only" なども選択可能
    bias = "none",
    
    # --- グラデーションチェックポイント ---
    # [NEW] "unsloth"を使用すると、VRAMを30%削減し、バッチサイズを2倍に拡大可能！
    # True: 標準的なグラデーションチェックポイント
    # "unsloth": Unsloth最適化版（非常に長いコンテキスト用）
    use_gradient_checkpointing = "unsloth",
    
    # --- ランダムシード ---
    # 再現性のための乱数シード
    random_state = 3407,
    
    # --- rsLoRA ---
    # ランク安定化LoRA（Rank Stabilized LoRA）のサポート
    use_rslora = False,
    
    # --- LoftQ ---
    # LoftQ（量子化を考慮したLoRA初期化）のサポート
    loftq_config = None,
)

### 現在のメモリ使用状況を表示

学習前のGPUメモリ使用状況を確認します。

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

### 最終メモリ使用量と学習時間の表示

学習後のメモリ使用状況と、学習にかかった時間を表示します。

## ステップ6: 推論（テスト） {#Inference}

モデルを実行してみましょう！指示（instruction）と入力（input）を変更できます。出力（output）は空白のままにしてください（モデルが生成します）。

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

### TextStreamerで継続的な推論

`TextStreamer`を使用すると、生成がトークンごとに表示されるため、待ち時間が短く感じられます。全体の完了を待つ必要がありません。

In [ ]:
# --- 必要なライブラリをインポート ---
from trl import SFTTrainer  # Supervised Fine-Tuning トレーナー
from transformers import TrainingArguments  # 学習パラメータの設定用
from unsloth import is_bfloat16_supported  # BFloat16がサポートされているか確認

# --- SFTTrainer（教師あり微調整トレーナー）の作成 ---
trainer = SFTTrainer(
    model = model,  # ファインチューニングするモデル
    tokenizer = tokenizer,  # トークナイザー（テキスト⇔トークンID変換）
    train_dataset = dataset,  # 学習用データセット
    dataset_text_field = "text",  # データセット内のテキストフィールド名
    max_seq_length = max_seq_length,  # 最大シーケンス長
    dataset_num_proc = 2,  # データセット処理の並列プロセス数
    
    # --- パッキング設定 ---
    # 短いシーケンスを1つのバッチにまとめて効率化
    # True: 短いシーケンスの場合、学習が5倍高速になる可能性
    # False: 各サンプルを個別に処理（長いシーケンスの場合はこちらを推奨）
    packing = False,
    
    # --- TrainingArguments（学習パラメータ） ---
    args = TrainingArguments(
        # --- バッチサイズ設定 ---
        # デバイスごとの学習バッチサイズ（GPU1つあたりのサンプル数）
        per_device_train_batch_size = 2,
        
        # --- 勾配累積ステップ ---
        # 実効バッチサイズ = per_device_train_batch_size × gradient_accumulation_steps
        # この場合: 2 × 4 = 8（メモリ不足を避けながら大きなバッチサイズを実現）
        gradient_accumulation_steps = 4,
        
        # --- ウォームアップステップ ---
        # 学習率を徐々に上げていくステップ数（学習初期の不安定性を軽減）
        warmup_steps = 5,
        
        # --- 学習エポック数 ---
        # num_train_epochs = 1,  # 完全な学習を行う場合はこちらを有効化
        
        # --- 最大ステップ数 ---
        # このノートブックでは高速化のため60ステップのみ実行
        # 完全な学習を行う場合は max_steps = None に設定
        max_steps = 60,
        
        # --- 学習率 ---
        # パラメータ更新の大きさを制御（2e-4 = 0.0002）
        learning_rate = 2e-4,
        
        # --- 精度設定 ---
        # fp16: Float16（16bit浮動小数点）を使用するか
        # bf16: BFloat16（Brain Float16）を使用するか
        # BFloat16が利用可能ならそちらを優先（Ampere世代以降のGPU）
        fp16 = not is_bfloat16_supported(),  # BFloat16が使えない場合にFloat16を使用
        bf16 = is_bfloat16_supported(),  # BFloat16が使える場合に使用
        
        # --- ロギング設定 ---
        # ロスなどの指標を記録する頻度（1ステップごと）
        logging_steps = 1,
        
        # --- オプティマイザー ---
        # adamw_8bit: AdamWの8bit版（メモリ効率的）
        optim = "adamw_8bit",
        
        # --- 重み減衰 ---
        # 過学習を防ぐための正則化パラメータ
        weight_decay = 0.01,
        
        # --- 学習率スケジューラー ---
        # "linear": 学習率を線形に減衰させる
        # その他: "cosine", "constant" なども選択可能
        lr_scheduler_type = "linear",
        
        # --- ランダムシード ---
        # 再現性のための乱数シード
        seed = 3407,
        
        # --- 出力ディレクトリ ---
        # チェックポイントやログの保存先
        output_dir = "outputs",
        
        # --- レポート設定 ---
        # WandB（Weights & Biases）などの実験トラッキングツールとの連携
        # "none": レポートしない（デフォルト）
        # "wandb": WandBにログを送信
        report_to = "none",
    ),
)

## ステップ7: ファインチューニングしたモデルの保存 {#Save}

最終モデルをLoRAアダプターとして保存するには、Hugging Faceの`push_to_hub`（オンライン保存）または`save_pretrained`（ローカル保存）を使用します。

**注意**: これはLoRAアダプターのみを保存し、完全なモデルではありません。16bitやGGUF形式で保存するには、下にスクロールしてください！

In [ ]:
# @title 現在のメモリ使用状況を表示

# --- GPUの情報を取得 ---
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 0（最初のGPU）の情報

# --- 学習開始前のメモリ使用量を記録 ---
# torch.cuda.max_memory_reserved(): これまでに予約された最大メモリ量（バイト）
# バイトをギガバイトに変換: / 1024 / 1024 / 1024
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

# --- GPUの最大メモリ容量 ---
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

# --- 情報を表示 ---
print(f"GPU = {gpu_stats.name}. 最大メモリ = {max_memory} GB.")
print(f"{start_gpu_memory} GB のメモリが予約されています。")

### 保存したLoRAアダプターの読み込み

保存したLoRAアダプターを推論用に読み込むには、以下の`False`を`True`に変更してください。

In [ ]:
# --- 学習の実行 ---
# trainer.train()を呼び出すと、設定したパラメータに基づいてファインチューニングが開始されます
# 戻り値のtrainer_statsには、学習時間、ロスの推移、メモリ使用量などの統計情報が含まれます
trainer_stats = trainer.train()

### Hugging Faceの標準的な方法での読み込み

Hugging Faceの`AutoModelForPeftCausalLM`を使用することもできます。ただし、`unsloth`がインストールされていない場合にのみ使用してください。

**注意**: この方法は非常に遅くなる可能性があります:
- 4bitモデルのダウンロードがサポートされていない
- Unslothの推論は2倍高速

したがって、可能な限りUnslothの使用を推奨します。

In [ ]:
# @title 最終メモリ使用量と学習時間の表示

# --- 学習後のメモリ使用量を計算 ---
# 現在の最大予約メモリ量（GB単位）
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

# LoRA学習で使用したメモリ量（学習後 - 学習前）
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)

# 最大メモリに対する使用率（パーセント）
used_percentage = round(used_memory / max_memory * 100, 3)

# 最大メモリに対するLoRA学習メモリの使用率（パーセント）
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# --- 統計情報の表示 ---
# 学習にかかった時間（秒）
print(f"{trainer_stats.metrics['train_runtime']} 秒で学習が完了しました。")

# 学習にかかった時間（分）
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} 分で学習が完了しました。")

# ピークメモリ使用量
print(f"ピーク予約メモリ = {used_memory} GB。")

# 学習に使用したメモリ量
print(f"学習に使用したピーク予約メモリ = {used_memory_for_lora} GB。")

# 最大メモリに対する使用率
print(f"ピーク予約メモリの最大メモリに対する割合 = {used_percentage} %。")

# 学習メモリの最大メモリに対する使用率
print(f"学習用ピーク予約メモリの最大メモリに対する割合 = {lora_percentage} %。")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
# alpaca_prompt = 上でコピーしたものを使用
# （データ準備セクションで定義したalpaca_promptテンプレートを使用）

# --- 推論モードを有効化 ---
# Unslothのネイティブ推論は、標準的な推論の2倍高速
FastLanguageModel.for_inference(model)

# --- 入力プロンプトの準備 ---
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.",  # instruction（指示）: フィボナッチ数列を続けてください
        "1, 1, 2, 3, 5, 8",  # input（入力）: 数列の初期値
        "",  # output（出力）: 空白のまま（モデルが生成します）
    )
], return_tensors = "pt").to("cuda")  # PyTorchテンソルとして返し、GPUに転送

# --- テキスト生成 ---
# model.generate()でテキストを生成
# max_new_tokens: 生成する最大トークン数
# use_cache: 高速化のためキャッシュを使用
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

# --- 生成されたトークンIDをテキストにデコード ---
# batch_decode()でトークンIDを人間が読めるテキストに変換
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = 上でコピーしたものを使用
# （データ準備セクションで定義したalpaca_promptテンプレートを使用）

# --- 推論モードを有効化 ---
# Unslothのネイティブ推論は、標準的な推論の2倍高速
FastLanguageModel.for_inference(model)

# --- 入力プロンプトの準備 ---
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.",  # instruction（指示）: フィボナッチ数列を続けてください
        "1, 1, 2, 3, 5, 8",  # input（入力）: 数列の初期値
        "",  # output（出力）: 空白のまま（モデルが生成します）
    )
], return_tensors = "pt").to("cuda")  # PyTorchテンソルとして返し、GPUに転送

# --- TextStreamerのインポートと作成 ---
from transformers import TextStreamer
# TextStreamer: トークンが生成されるたびにリアルタイムで表示
# 全体の生成完了を待たずに、生成過程を確認できるため待ち時間が短く感じられます
text_streamer = TextStreamer(tokenizer)

# --- ストリーミング生成 ---
# streamer引数にTextStreamerを渡すことで、トークンごとに表示されます
# max_new_tokens: 生成する最大トークン数（128トークン）
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# --- LoRAアダプターのローカル保存 ---
# モデル（LoRAアダプターのみ）を"lora_model"ディレクトリに保存
model.save_pretrained("lora_model")

# トークナイザーも同じディレクトリに保存
tokenizer.save_pretrained("lora_model")

# --- オンライン保存（Hugging Face Hub） ---
# Hugging Face Hubにアップロードする場合は、以下のコメントを外してください
# "your_name"を自分のユーザー名に変更し、トークン（token）を設定する必要があります
# model.push_to_hub("your_name/lora_model", token = "...")  # モデルをオンラインに保存
# tokenizer.push_to_hub("your_name/lora_model", token = "...")  # トークナイザーをオンラインに保存

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
# --- 保存したLoRAアダプターの読み込み ---
# 以下のコードを使用する場合は、if False を if True に変更してください
if False:
    from unsloth import FastLanguageModel
    
    # 保存したLoRAアダプターを読み込む
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",  # 保存したモデルのパス（学習に使用したモデル）
        max_seq_length = max_seq_length,  # 最大シーケンス長
        dtype = dtype,  # データ型
        load_in_4bit = load_in_4bit,  # 4bit量子化を使用
    )
    
    # 推論モードを有効化（Unslothのネイティブ推論は2倍高速）
    FastLanguageModel.for_inference(model)

# --- alpaca_promptの準備 ---
# 注意: alpaca_promptは上のデータ準備セクションからコピーする必要があります！

# --- 入力プロンプトの準備 ---
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?",  # instruction（指示）: パリの有名な高い塔は何ですか？
        "",  # input（入力）: 空白
        "",  # output（出力）: 空白のまま（モデルが生成します）
    )
], return_tensors = "pt").to("cuda")  # PyTorchテンソルとして返し、GPUに転送

# --- TextStreamerで生成 ---
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)  # トークンごとにリアルタイムで表示
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)  # 128トークンまで生成

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.